# Extract data from Wikipedia

I read the table from wikipedia

In [2]:
import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

I convert the read table into a DataFrame

In [47]:
right_table=soup.find('table', class_='wikitable sortable')

A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
import pandas as pd
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df['PostalCode'] = df['PostalCode'].astype(str)
df['Borough'] = df['Borough'].astype(str)
df['Neighborhood'] = df['Neighborhood'].astype(str)
df['PostalCode']=df['PostalCode'].map(lambda x: x.lstrip('\n').rstrip('\n'))
df['Borough']=df['Borough'].map(lambda x: x.lstrip('\n').rstrip('\n'))
df['Neighborhood']=df['Neighborhood'].map(lambda x: x.lstrip('\n').rstrip('\n'))
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


I cancel the 'Not assigned' for the column Borough

In [46]:
df=df[(df['Borough'] != 'Not assigned')]

The neighborhoods were already joined base on the postal code area when the database was downloaded by wikipedia

I check that there aren't 'Not assigned' values for the Neighborhood column

In [42]:
dc = df[(df['Neighborhood'] == 'Not assigned')]
dc.shape
#there aren't

(0, 3)

DataFrame dimension

In [45]:
df.shape

(103, 3)

# Read latitude and longitude

I read the data from the csv

In [51]:
df_ll = pd.read_csv('http://cocl.us/Geospatial_data')
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


I merge the tables and i drop the duplicated column

In [57]:
result = df.merge(df_ll, left_on='PostalCode', right_on='Postal Code')
result.drop(columns=['Postal Code'])
result.shape

(103, 6)